In [34]:
!apt-get update && apt-get install -y wget ffmpeg
!pip install pandas

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease              
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease                        
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
wget is already the newest version (1.19.4-1ubuntu2.2).
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 81 not upgraded.
     |███

In [42]:
!ls -lh /experiment_dir/dataset/

total 1.1G
-rw-r--r-- 1 root root 5.3M May 19  2011 TownCentre-groundtruth.top
-rw-r--r-- 1 root root 141M Oct 30  2011 TownCentreXVID.avi
drwxr-xr-x 2 root root 120K May 24 11:25 images
-rw-r--r-- 1 root root 651M May 24 11:37 oxford_town_center_train.record
-rw-r--r-- 1 root root 291M May 24 11:37 oxford_town_center_val.record
drwxr-xr-x 2 root root 124K May 24 11:34 xmls


In [43]:
%set_env DATASET_DIR=/experiment_dir/dataset
%set_env PRETRAINED_MODELS_DIR =/experiment_dir/pretrained_models

env: DATASET_DIR=/experiment_dir/dataset
env: PRETRAINED_MODELS_DIR=/experiment_dir/pretrained_models


In [9]:
# Download Oxford Town Center Dataset video file and csv annotation file
!bash ./download_towncenter_video_and_labels.sh

--2020-05-24 11:16:21--  http://www.robots.ox.ac.uk/ActiveVision/Research/Projects/2009bbenfold_headpose/Datasets/TownCentreXVID.avi
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 147486436 (141M) [video/x-msvideo]
Saving to: ‘/experiment_dir/dataset/TownCentreXVID.avi’

/experiment_dir/dat 100%[===================>] 140.65M  9.07MB/s    in 40s     

2020-05-24 11:17:01 (3.56 MB/s) - ‘/experiment_dir/dataset/TownCentreXVID.avi’ saved [147486436/147486436]

--2020-05-24 11:17:01--  https://www.robots.ox.ac.uk/ActiveVision/Research/Projects/2009bbenfold_headpose/Datasets/TownCentre-groundtruth.top
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5553300 (5.3M)
Saving to: ‘

In [17]:
# Extract images from video, resize them and save in images directory with `ffmpeg`
!mkdir $DATASET_DIR/images
!ffmpeg -i $DATASET_DIR/TownCentreXVID.avi -q:v 1 -start_number 0 -frames:v 4501 $DATASET_DIR/images/%d.jpg

mkdir: cannot create directory ‘/experiment_dir/dataset/images’: File exists
ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavp

In [39]:
# Create xml annotation file and save in xmls directory
!python create_xmls.py --annotation_path $DATASET_DIR/TownCentre-groundtruth.top

==================== Start Creating xml Files! ====================


In [40]:
!python create_tfrecord.py --data_dir $DATASET_DIR --output_dir $DATASET_DIR --label_map_path ./label_map.pbtxt



W0524 11:37:08.788470 140503053051712 module_wrapper.py:139] From /models/research/object_detection/utils/label_map_util.py:138: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

I0524 11:37:08.788950 140503053051712 create_tfrecord.py:168] Reading from Oxford Town Center Dataset.
I0524 11:37:08.790720 140503053051712 create_tfrecord.py:182] 3150 training and 1351 validation examples.

W0524 11:37:08.790819 140503053051712 module_wrapper.py:139] From create_tfrecord.py:138: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.

I0524 11:37:08.790948 140503053051712 create_tfrecord.py:141] On image 0 of 3150
/models/research/object_detection/utils/dataset_util.py:79: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if not xml:
I0524 11:37:08.882479 140503053051712 create_tfrecord.py:141] On image 100 of 3150
I0524 11:37:08.968871 1

In [45]:
!mkdir $PRETRAINED_MODELS_DIR

In [55]:
!ls  -lh $PRETRAINED_MODELS_DIR

total 180M
drwxr-xr-x 3 345018 89939 4.0K Mar 30  2018 ssd_mobilenet_v2_coco_2018_03_29
-rw-r--r-- 1 root   root  180M Apr  2  2018 ssd_mobilenet_v2_coco_2018_03_29.tar.gz


In [48]:
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz -O $PRETRAINED_MODELS_DIR/ssd_mobilenet_v2_coco_2018_03_29.tar.gz
!wget http://download.tensorflow.org/models/object_detection/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz -O $PRETRAINED_MODELS_DIR/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz
!wget http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet50_coco_2018_01_28.tar.gz -O $PRETRAINED_MODELS_DIR/faster_rcnn_resnet50_coco_2018_01_28.tar.gz

--2020-05-24 11:54:56--  http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.5.112, 2607:f8b0:4005:808::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.5.112|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 187925923 (179M) [application/x-tar]
Saving to: ‘/experiment_dir/pretrained_models/ssd_mobilenet_v2_coco_2018_03_29.tar.gz’

/experiment_dir/pre 100%[===================>] 179.22M  16.4MB/s    in 11s     

2020-05-24 11:55:07 (16.6 MB/s) - ‘/experiment_dir/pretrained_models/ssd_mobilenet_v2_coco_2018_03_29.tar.gz’ saved [187925923/187925923]



In [53]:
!tar -xzvf $PRETRAINED_MODELS_DIR/ssd_mobilenet_v2_coco_2018_03_29.tar.gz -C $PRETRAINED_MODELS_DIR

ssd_mobilenet_v2_coco_2018_03_29/checkpoint
ssd_mobilenet_v2_coco_2018_03_29/model.ckpt.meta
ssd_mobilenet_v2_coco_2018_03_29/pipeline.config
ssd_mobilenet_v2_coco_2018_03_29/saved_model/saved_model.pb
ssd_mobilenet_v2_coco_2018_03_29/frozen_inference_graph.pb
ssd_mobilenet_v2_coco_2018_03_29/saved_model/
ssd_mobilenet_v2_coco_2018_03_29/saved_model/variables/
ssd_mobilenet_v2_coco_2018_03_29/model.ckpt.index
ssd_mobilenet_v2_coco_2018_03_29/
ssd_mobilenet_v2_coco_2018_03_29/model.ckpt.data-00000-of-00001


In [59]:
!mkdir /experiment_dir/ped_ssd_mobilenet_v2
%set_env PIPELINE_CONFIG_PATH=/repo/training/tf_object_detection_api/pedestrian_detection/configs/ssd_mobilenet_v2_pedestrian.config
%set_env MODEL_DIR=/experiment_dir/ped_ssd_mobilenet_v2
%set_env NUM_TRAIN_STEPS=50000
%set_env SAMPLE_1_OF_N_EVAL_EXAMPLES=1

env: PIPELINE_CONFIG_PATH=/repo/training/tf_object_detection_api/pedestrian_detection/configs/ssd_mobilenet_v2_pedestrian.config
env: MODEL_DIR=/experiment_dir/ped_ssd_mobilenet_v2
env: NUM_TRAIN_STEPS=50000
env: SAMPLE_1_OF_N_EVAL_EXAMPLES=1


In [ ]:
!python /models/research/object_detection/model_main.py --pipeline_config_path=${PIPELINE_CONFIG_PATH} --model_dir=${MODEL_DIR} --num_train_steps=${NUM_TRAIN_STEPS} --sample_1_of_n_eval_examples=$SAMPLE_1_OF_N_EVAL_EXAMPLES --alsologtostderr

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0524 12:19:17.108064 140528213337920 module_wrapper.py:139] From /models/research/object_detection/utils/config_util.py:102: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0524 12:19:17.110463 140528213337920 module_wrapper.py:139] From /models/research/object_detection/model_lib.py:628: The name tf.logging.warning is deprecated. Please use tf.compat.v1.logging.warning instead.

W0524 12:19:17.110527 140528213337920 model_lib.py:629] Forced number of epochs for all eval validations to be 1.

W0524 12:19:17.110599 140528213337920 module_wrapper.py:139] From /models/research/object_detection/uti


W0524 12:19:18.218487 140528213337920 module_wrapper.py:139] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/autograph/converters/directives.py:119: The name tf.logging.warn is deprecated. Please use tf.compat.v1.logging.warn instead.


W0524 12:19:24.139538 140528213337920 module_wrapper.py:139] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/autograph/converters/directives.py:119: The name tf.is_nan is deprecated. Please use tf.math.is_nan instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0524 12:19:24.205282 140528213337920 deprecation.py:323] From /models/research/object_detection/utils/ops.py:493: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

W0524 12:19:26.056770 140528213337920 module_wrapper.py:139] From /usr/local/lib/python3.6/dist-packa

W0524 12:19:35.578225 140528213337920 variables_helper.py:154] Variable [FeatureExtractor/MobilenetV2/layer_19_2_Conv2d_2_3x3_s2_512/weights] is available in checkpoint, but has an incompatible shape with model variable. Checkpoint shape: [[1, 1, 256, 512]], model variable shape: [[3, 3, 256, 512]]. This variable will not be initialized from the checkpoint.
W0524 12:19:35.578315 140528213337920 variables_helper.py:154] Variable [FeatureExtractor/MobilenetV2/layer_19_2_Conv2d_3_3x3_s2_256/weights] is available in checkpoint, but has an incompatible shape with model variable. Checkpoint shape: [[1, 1, 128, 256]], model variable shape: [[3, 3, 128, 256]]. This variable will not be initialized from the checkpoint.
W0524 12:19:35.578385 140528213337920 variables_helper.py:154] Variable [FeatureExtractor/MobilenetV2/layer_19_2_Conv2d_4_3x3_s2_256/weights] is available in checkpoint, but has an incompatible shape with model variable. Checkpoint shape: [[1, 1, 128, 256]], model variable shape:

2020-05-24 12:19:47.123208: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-05-24 12:19:47.322188: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-05-24 12:19:47.331255: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-05-24 12:19:47.331819: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x513d8f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2020-05-24 12:19:47.331836: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): GeForce RTX 2070 SUPER, Compute Capability 7.5
2020-05-24 12:19:47.331843: I tensorflow/compiler/xla/service/service.cc:176]

In [62]:
from object_detection import model_main

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



<module 'object_detection.model_main' from '/models/research/object_detection/model_main.py'>